In [158]:
#Handle Imports
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import os
import json
import pprint
import io
import requests
from PIL import Image
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import uuid
from ollama import chat
from ollama import ChatResponse
from random import randrange
import os
import glob
import shutil


#load from .env file which should exist
load_dotenv()

#initialize spotipy client
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)


In [171]:
#given an image performs the processing

def clear_images():
   for hgx in glob.glob("/Users/MurphyD1/Documents/Python/sloppa/images/*"):
       print(hgx)
      # os.remove(hgx)
    

def slop_generator(track, artist, star):
    response: ChatResponse = chat(model='llama3.2', messages=[
      {
        'role': 'user',
        'content': f"""Generate some vague but smart-sounding commentary on the track {track} by {artist}. Should be indicative of a {star} star review. 20 words max and should be understandable by a common person""",
      },
    ])
    return response['message']['content']


def download_image(link):
    url = 'https://cdn.pixabay.com/photo/2023/11/16/05/02/mountains-8391433_640.jpg'
    response = requests.get(url)
    file_path = str(uuid.uuid4()) + ".jpg"
    if response.status_code == 200:
        with open("images/" + file_path, 'wb') as file:
            file.write(response.content)
    return file_path


#given an image performs the processing
def process_image(filename):
    original_img = Image.open(filename)
    resized_img = original_image.resize((1080,1080), Image.LANCZOS)
    #following standard phone/tiktok dimensions
    new_wdith, new_height = 1080,1920
    color = (166,123,91)
    background = Image.new("RGB", (new_width, new_height), color)
    x_offset = 0
    y_offset = (1920 - 1080) // 2
    background.paste(resized_image, (x_offset, y_offset))
    new_filename = filename.replace(".jpg","") + "_p" + ".jpg"
    background.save(new_filename)

#given a playlist builds a dictionary of items 

def load_items(playlist):
    track_list = []
    result = sp.playlist(playlist, fields=None, market=None, additional_types=('track',))
    tracks = result['tracks']['items']
    tracks = tracks[0:5]
    
    for track in tracks:
        
        #extract track information 
        track = track['track']
        name = track['name']
        artists = track['artists']
        image = track['album']['images'][0]['url']
        artist  = (artists[0]['name'])
        
        #download image
        res = download_image(image)

        #assign random star rating and generate text slop for it
        stars = randrange(1,5)
        slop = slop_generator(name, artist, stars)

        #print(name, artist, image)
        d = {
            "name": name,
            "artist": artist,
            "image": image,
            "image_id": res,
            "analysis": slop
        }
        track_list.append(d)

        #res = download_image(image)
    return track_list  

In [175]:

items = load_items('spotify:playlist:0VLYIYEEt27T8cPc2EGogY')
#pprint.p(items)

In [174]:
clear_images()